In [ ]:
#
mov reg, [esp+X]        ; input
add/shl/lea reg, offset ; arithmetic
cmp / test reg, limit   ; (optional, often weak or missing)
mov [mem], reg          ; write to buffer
(call strcpy / memcpy)  ; function call, classic overflow

In [2]:
#!/usr/bin/env python3
"""
Buffer Overflow Template Generator

This notebook generates comprehensive templates for detecting buffer overflow vulnerabilities
based on analysis of real buffer overflow patterns found in the satellite_ground_station.c
vulnerability research.

Key findings:
- Primary pattern: lea.X1;movzx;sub;movsxd (stack buffer + size manipulation + extension)
- Found 3 vulnerabilities: B(100000d71), B(100000f29), B(100001055)
- All follow: lea -> mov -> add -> movzx -> sub -> movsxd -> memcpy call
"""

import itertools
from datetime import datetime

print("=== Buffer Overflow Template Generator ===")
print(f"Generated: {datetime.now().isoformat()}")
print()

# Based on our findings, the core buffer overflow pattern involves:
# 1. BUFFER_SETUP: Stack buffer allocation (lea)
# 2. SIZE_LOAD: Loading user-controlled size (movzx)  
# 3. SIZE_MANIP: Arithmetic on size (sub, add)
# 4. SIZE_EXTEND: Sign/zero extension (movsxd, movzwl)
# 5. DANGEROUS_OP: Call to dangerous function (call) or memory operation (mov)

# Define instruction categories for buffer overflow detection
BUFFER_SETUP = ["lea", "mov", "push"]
SIZE_LOAD = ["movzx", "movzwl", "mov"]  
SIZE_MANIP = ["sub", "add", "shl", "shr", "mul", "imul"]
SIZE_EXTEND = ["movsxd", "movzx", "movslq"]
DANGEROUS_OP = ["call", "mov"]
MEMORY_OPS = ["mov", "lea", "add"]  # Instructions that can manipulate pointers

print("1. CORE BUFFER OVERFLOW PATTERNS")
print("=" * 50)

# Our confirmed working pattern
core_patterns = [
    "lea.X1;movzx;sub;movsxd",
    "lea.X1;movzx;sub;movsxd;call",
]

print("Confirmed vulnerability patterns:")
for pattern in core_patterns:
    print(f"  {pattern}")

print()
print("2. SYSTEMATIC BUFFER OVERFLOW TEMPLATES")
print("=" * 50)

# Generate systematic buffer overflow patterns
buffer_overflow_templates = []

# Pattern 1: Buffer setup + size manipulation + dangerous operation
for buffer_op in BUFFER_SETUP:
    for size_op in SIZE_LOAD:
        for manip_op in SIZE_MANIP:
            for extend_op in SIZE_EXTEND:
                # Basic pattern with wildcards between each step
                pattern1 = f"{buffer_op}.X1;{size_op}.X2;{manip_op}.X3;{extend_op}"
                pattern2 = f"{buffer_op}.X1;{size_op}.X2;{manip_op}.X3;{extend_op}.X4;call"
                buffer_overflow_templates.extend([pattern1, pattern2])

# Pattern 2: Simplified patterns (more permissive)
for buffer_op in BUFFER_SETUP:
    for size_op in SIZE_LOAD:
        for manip_op in SIZE_MANIP:
            # Simplified patterns
            pattern1 = f"{buffer_op}.X1;{size_op}.X2;{manip_op}"
            pattern2 = f"{buffer_op}.X1;{size_op}.X2;{manip_op}.X3;call"
            buffer_overflow_templates.extend([pattern1, pattern2])

# Pattern 3: Direct size manipulation patterns
for size_op in SIZE_LOAD:
    for manip_op in SIZE_MANIP:
        pattern1 = f"{size_op}.X1;{manip_op}"
        pattern2 = f"{size_op}.X1;{manip_op}.X2;call"
        buffer_overflow_templates.extend([pattern1, pattern2])

# Remove duplicates while preserving order
seen = set()
unique_templates = []
for template in buffer_overflow_templates:
    if template not in seen:
        seen.add(template)
        unique_templates.append(template)

print(f"Generated {len(unique_templates)} unique buffer overflow templates:")
print()

# Group templates by complexity
simple_patterns = [t for t in unique_templates if t.count(';') <= 1]
medium_patterns = [t for t in unique_templates if t.count(';') == 2]
complex_patterns = [t for t in unique_templates if t.count(';') >= 3]

print("SIMPLE PATTERNS (2-3 instructions):")
for i, pattern in enumerate(simple_patterns[:20], 1):  # Show first 20
    print(f"  {i:2d}. {pattern}")
if len(simple_patterns) > 20:
    print(f"  ... and {len(simple_patterns) - 20} more")

print()
print("MEDIUM PATTERNS (3-4 instructions):")
for i, pattern in enumerate(medium_patterns[:20], 1):  # Show first 20
    print(f"  {i:2d}. {pattern}")
if len(medium_patterns) > 20:
    print(f"  ... and {len(medium_patterns) - 20} more")

print()
print("COMPLEX PATTERNS (4+ instructions):")
for i, pattern in enumerate(complex_patterns[:20], 1):  # Show first 20
    print(f"  {i:2d}. {pattern}")
if len(complex_patterns) > 20:
    print(f"  ... and {len(complex_patterns) - 20} more")

print()
print("3. HIGH-PRIORITY BUFFER OVERFLOW TEMPLATES")
print("=" * 50)

# High-priority patterns based on our vulnerability analysis
high_priority = [
    # Our exact findings
    "lea.X1;movzx;sub;movsxd",
    "lea.X1;movzx;sub;movsxd;call",
    
    # Variations of our findings
    "lea.X1;mov.X2;movzx;sub;movsxd",
    "lea.X1;add.X2;movzx;sub;movsxd",
    "lea.X1;movzx;sub;movsxd;mov.X2;call",
    
    # Common buffer overflow patterns
    "lea.X1;movzx;call",           # Direct buffer + size + call
    "movzx.X1;sub.X2;call",        # Size manipulation + call
    "lea.X1;sub.X2;call",          # Buffer + size manipulation + call
    "mov.X1;movzx.X2;sub.X3;call", # Load + size + manipulation + call
    
    # Stack-based patterns
    "push.X1;movzx.X2;sub.X3;call",
    "lea.X1;movzx.X2;add.X3;call",
    "lea.X1;movzx.X2;shl.X3;call",
    
    # Extension patterns
    "movzx.X1;movsxd.X2;call",
    "lea.X1;movzx.X2;movsxd.X3;call",
]

print("High-priority templates for vulnerability detection:")
for i, pattern in enumerate(high_priority, 1):
    print(f"  {i:2d}. {pattern}")

print()
print("4. TEMPLATE USAGE GUIDE")
print("=" * 50)

usage_guide = """
USAGE WITH SLICER:
  python3 slicer_fixed.py behavior_algebra.txt "<template>" angr_asm_dump.txt

RECOMMENDED TESTING ORDER:
1. Start with high-priority templates (most likely to find vulnerabilities)
2. Test core patterns first: "lea.X1;movzx;sub;movsxd"
3. Then expand to variations with calls: "lea.X1;movzx;sub;movsxd;call"
4. Test simpler patterns for broader coverage: "movzx.X1;sub.X2;call"

PATTERN EXPLANATION:
- lea.X1    : Stack buffer allocation with wildcard gap
- movzx     : Load user-controlled size (16-bit to 32-bit)
- sub       : Size manipulation (dangerous arithmetic)
- movsxd    : Sign extend to 64-bit (preparation for memcpy)
- call      : Function call (memcpy, strcpy, etc.)

WILDCARDS (.X1, .X2, etc.):
- Allow any instructions between pattern elements
- Essential for real-world matching (compilers insert extra instructions)
- Example: lea.X1;movzx matches "lea -> mov -> add -> movzx"
"""

print(usage_guide)

print()
print("5. EXPORT ALL TEMPLATES")
print("=" * 50)

# Export all templates to a file for batch testing
all_templates = high_priority + unique_templates

# Remove duplicates from final list
final_templates = []
seen_final = set()
for template in all_templates:
    if template not in seen_final:
        seen_final.add(template)
        final_templates.append(template)

print(f"Total unique templates: {len(final_templates)}")

# Save to file
output_file = f"buffer_overflow_templates_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(output_file, 'w') as f:
    f.write("# Buffer Overflow Detection Templates\n")
    f.write(f"# Generated: {datetime.now().isoformat()}\n")
    f.write("# Usage: python3 slicer_fixed.py behavior_algebra.txt \"<template>\" angr_asm_dump.txt\n")
    f.write("#\n")
    f.write("# HIGH PRIORITY TEMPLATES (test these first):\n")
    for i, template in enumerate(high_priority, 1):
        f.write(f"# {i:2d}. {template}\n")
    f.write("#\n")
    f.write("# ALL TEMPLATES:\n")
    for i, template in enumerate(final_templates, 1):
        f.write(f"{template}\n")

print(f"Templates saved to: {output_file}")
print()
print("READY FOR VULNERABILITY DETECTION!")
print("Use the generated templates with slicer_fixed.py to detect buffer overflow vulnerabilities.")

=== Buffer Overflow Template Generator ===
Generated: 2025-07-11T18:24:22.730849

1. CORE BUFFER OVERFLOW PATTERNS
Confirmed vulnerability patterns:
  lea.X1;movzx;sub;movsxd
  lea.X1;movzx;sub;movsxd;call

2. SYSTEMATIC BUFFER OVERFLOW TEMPLATES
Generated 468 unique buffer overflow templates:

SIMPLE PATTERNS (2-3 instructions):
   1. movzx.X1;sub
   2. movzx.X1;add
   3. movzx.X1;shl
   4. movzx.X1;shr
   5. movzx.X1;mul
   6. movzx.X1;imul
   7. movzwl.X1;sub
   8. movzwl.X1;add
   9. movzwl.X1;shl
  10. movzwl.X1;shr
  11. movzwl.X1;mul
  12. movzwl.X1;imul
  13. mov.X1;sub
  14. mov.X1;add
  15. mov.X1;shl
  16. mov.X1;shr
  17. mov.X1;mul
  18. mov.X1;imul

MEDIUM PATTERNS (3-4 instructions):
   1. lea.X1;movzx.X2;sub
   2. lea.X1;movzx.X2;add
   3. lea.X1;movzx.X2;shl
   4. lea.X1;movzx.X2;shr
   5. lea.X1;movzx.X2;mul
   6. lea.X1;movzx.X2;imul
   7. lea.X1;movzwl.X2;sub
   8. lea.X1;movzwl.X2;add
   9. lea.X1;movzwl.X2;shl
  10. lea.X1;movzwl.X2;shr
  11. lea.X1;movzwl.X2;mul


In [ ]:
Primary Buffer Overflow Pattern:
  "lea;movzwl;subl;movslq;call"

  Alternative Patterns:
  "subq;movzwl;subl;call"
  "lea;movzwl;call"
  "movzwl;subl;call"
  "lea;subl;call"

  Most Critical Pattern (from your crash):
  "lea;movzwl;subl;movslq;call"

  This sequence captures:
  - lea - stack buffer setup
  - movzwl - load user-controlled size (16-bit)
  - subl - arithmetic on size (dangerous)
  - movslq - sign extend size to 64-bit
  - call - dangerous function (memcpy/strcpy)

  The pattern "lea;movzwl;subl;movslq;call" will find the exact vulnerability that caused your segfault at line 130 in process_attitude_control.
